In [14]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time

In [15]:
import cv2
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from tensorflow.keras.callbacks import *

In [16]:
imagePaths = []
for dirname, _, filenames in os.walk('Resultado GAN'):
    for filename in filenames:
        imagePaths.append(os.path.join(dirname, filename))

In [17]:
 len(imagePaths)

1776

In [18]:
 imagePaths[0]

'Resultado GAN\\healthy\\Generated\\1.tiff'

In [19]:
#Taking the images from the folder and resizing to 128x128 RGB
#labeling them from ['healthy', 'mild', 'moderate', 'severe'] to -> [0,1,2,3]
new_img_height = 128
new_img_width = 128
channels = 3
all_labels = ['healthy', 'mild', 'moderate', 'severe']
original_data_x = []
original_data_y = []
for imagePath in tqdm(imagePaths):
    if imagePath.split(os.path.sep)[-2] != 'Generated':
        continue
    label = all_labels.index(imagePath.split(os.path.sep)[-3])
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (new_img_height, new_img_width))
    original_data_x.append(image)
    original_data_y.append(label)

100%|███████████████████████████████████████████████████████████████████████| 1776/1776 [00:00<00:00, 2866.93it/s]


In [20]:
imagePaths = []

In [21]:
import sys

In [22]:
print("qtd = ", len(all_data))

qtd =  592


In [ ]:
np.save('data_x_original.npy', original_data_x)
np.save('data_y_original.npy', original_data_y)

## **Image Cropping**

In [24]:
def cutting_image(image, image_height, image_width, window_size, window_cut):
    windows = []
    for y in range(0, image_height, window_cut):
        for x in range(0, image_width, window_cut):
            window = image[y:y+window_size, x:x+window_size]
            windows.append(window)
    return windows

In [25]:
#import cv2 

In [26]:
# old_img_height = 128
# old_img_width = 128
# new_img_height = 32
# new_img_width = 32
# channels = 3
# new_data_x = []
# new_data_y = []
# for i in range(len(data_x)):
#  image = data_x[i]
#  #im = np.transpose(image, (1, 2, 0))
#  #image = tf.image.resize(image, (new_img_height,new_img_width))
#  #d.csv.iloc[i]["finding"]
#  label = data_y[i]
#  cutted_images = cutting_image(image, old_img_height, old_img_width, new_img_height, new_img_width)
#  for new_image in cutted_images:
#    #new_im = tf.image.resize(new_image, (new_img_height,new_img_width))
#    #new_im = tf.image.per_image_standardization(new_im)
#    #new_im = cv2.cvtColor(new_im, cv2.COLOR_BGR2RGB)
#    new_data_x.append(new_image)
#    new_data_y.append(label)

In [27]:
# data_x = new_data_x
# data_y = new_data_y

In [28]:
# print("Quantidade de imagens:", len(data_x))

# **Testando imagens**

In [29]:
# fig = plt.figure(figsize=(5, 5))
# for i in range(1, (old_img_height//new_img_height)*(old_img_width//new_img_width)+1):
#     fig.add_subplot((old_img_height//new_img_height), (old_img_width//new_img_width), i)
#     #im = np.transpose(data_x[i - 1], (2, 0, 1))
#     #im = data_x[i-1]
#     plt.imshow(data_x[i-1])
#     plt.axis('off')
#     fig.tight_layout(pad=0.2)

In [30]:
# plt.imshow(data_x[0], cmap="gray");

# **Data Augmentation**



In [31]:
#cria cópias com rotação de 90 graus
rot_x = []
rot_y = []
for i in range(len(original_data_x)):
    rot_x.append(cv2.rotate(original_data_x[i], cv2.ROTATE_90_COUNTERCLOCKWISE))
    rot_y.append(original_data_y[i])

In [35]:
#cria cópias espelhadas horizontalmente
flip1_x = []
flip1_y = []
for i in range(len(original_data_x)):
    flip1_x.append(cv2.flip(original_data_x[i], 1))
    flip1_y.append(original_data_y[i])

In [36]:
#cria cópias espelhadas verticalmente
flip2_x = []
flip2_y = []
for i in range(len(original_data_x)):
    flip2_x.append(cv2.flip(original_data_x[i], 0))
    flip2_y.append(original_data_y[i])

In [38]:
#concatena as imagens formadas no array principal
new_data_x = [*original_data_x,*rot_x,*flip1_x, *flip2_x]
new_data_y = [*original_data_y,*rot_y,*flip1_y, *flip2_y]

In [39]:
print('Data_x size:', len(new_data_x))
print('Data label size:', len(new_data_y))

Data_x size: 2368
Data label size: 2368


In [40]:
import pandas as pd

In [42]:
pd.Series(new_data_y).value_counts()

0    592
1    592
2    592
3    592
Name: count, dtype: int64

In [48]:
# fig = plt.figure(figsize=(5, 5))
# for i in range(1, (old_img_height//new_img_height)*(old_img_width//new_img_width)+1):
#     fig.add_subplot((old_img_height//new_img_height), (old_img_width//new_img_width), i)
#     #im = np.transpose(data_x[i - 1], (2, 0, 1))
#     #im = data_x[i-1]
#     plt.imshow(data_x[i-1])
#     plt.axis('off')
#     fig.tight_layout(pad=0.2)

In [56]:
import random
def shuffle_dataset(data_x, data_y):
    new_data = []
    for i in range(len(data_x)):
        new_data.append((data_x[i], data_y[i]))
    random.shuffle(new_data)
    new_data_x = []
    new_data_y = []
    for i in range(len(new_data)):
        new_data_x.append(new_data[i][0])
        new_data_y.append(new_data[i][1])
    return new_data_x, new_data_y

In [57]:
#shuffling the final dataset
new_data_x, new_data_y = shuffle_dataset(new_data_x, new_data_y)

In [58]:
len(new_data_x), len(new_data_y)

(2368, 2368)

In [59]:
pd.Series(new_data_y).value_counts()

1    592
3    592
0    592
2    592
Name: count, dtype: int64

In [60]:
np.save('data_x_aumentated.npy', new_data_x)
np.save('data_y_aumentated.npy', new_data_y)

In [63]:
new_data_x = [i/255.0 for i in new_data_x]
np.save('data_x_aumentated_normalized.npy', new_data_x)
np.save('data_y_aumentated_normalized.npy', new_data_y)

In [64]:
new_data_x[0]

array([[[0.65098039, 0.58431373, 0.68627451],
        [0.69803922, 0.63137255, 0.72156863],
        [0.69803922, 0.64705882, 0.7372549 ],
        ...,
        [0.58039216, 0.49411765, 0.65490196],
        [0.57254902, 0.49411765, 0.64705882],
        [0.5372549 , 0.47058824, 0.62352941]],

       [[0.68627451, 0.61176471, 0.72156863],
        [0.70196078, 0.62745098, 0.73333333],
        [0.70588235, 0.62352941, 0.7254902 ],
        ...,
        [0.60392157, 0.49803922, 0.65882353],
        [0.6       , 0.49803922, 0.66666667],
        [0.58431373, 0.49411765, 0.65882353]],

       [[0.6745098 , 0.60784314, 0.71764706],
        [0.69411765, 0.62352941, 0.70980392],
        [0.69803922, 0.63137255, 0.71764706],
        ...,
        [0.61568627, 0.51372549, 0.67058824],
        [0.61176471, 0.52156863, 0.6745098 ],
        [0.58823529, 0.50980392, 0.65490196]],

       ...,

       [[0.64313725, 0.55294118, 0.69803922],
        [0.60784314, 0.51764706, 0.65490196],
        [0.63921569, 0